# The secretary problem

Given a set of applicants, we have to select the best one. There's limited time for the task, though, and we must balance a good choice and the time invested in the search. Instead of analysing each applicant, we should check around 37% of them and then get the following one that exceed the best one out of the 37% analised.

Here I'll compare the result of such technique against analysing each single applicant.

In [4]:
from random import sample

def optimal_percentage(qt_cand):
    '''
    Returns the optimal percentage of the sample that should be used for the 37% rule
    INPUT: quantity of candidates
    OUTPUT: percentage of  the sample
    '''
    table = {1:100,2:50,3:33.33,4:25,5:40,6:33.33,7:28.57,8:37.5,9:33.33,
             10:30,20:35,30:36.67,40:37.5,50:36,100:37,1000:36.9}
    
    #Higher than 1000? Fixed value.
    if int(qt_cand) > 1000:
        return 36.9
    
    #Between 1 and 1000? Looks for where it matches in between the defined values.
    keys = list(table.keys())
    for key in keys:
        if key == qt_cand:
            return table[key]
        elif key < qt_cand:
            prev_key = key
            continue
        else:
            return table[prev_key]

    
def opt_stop_37(candidates):
    '''
    Uses the 37% rule to select a possible best candidate.
    INPUT: list of candidates
    OUTPUT: selected candidate
    '''
    qt_cand = len(candidates)
    
    #Deciding the optimal sampling percentage.
    sample_perc = optimal_percentage(int(qt_cand))
    
    #Now let's read the whole sample and get the best candidate out of it
    perc_read = float()
    qt_read = int()
    best_sample = int()
    
    while(perc_read < sample_perc):
        if candidates[qt_read] > best_sample:
            best_sample = candidates[qt_read]
        
        qt_read += 1
        perc_read = (qt_read / int(qt_cand)) * 100
        
    #Let's test the rest of the candidates and get the first one that is better the best sampled one.
    #NOTE: in case the best of all candidates was in the sample, we'll have to get the best among the
    #remaining candidates, even it being worse than the best sampled one.
    best_def = int()
    for pos,cand in enumerate(candidates):
        #Skipping the sampled ones
        if qt_read > pos: #it's not >= because pos starts in 0
            continue
        
        #Found a better than the best sampled.
        if cand > best_sample:
            return cand
        #Found the best so far which still is worse than the best sampled
        elif best_def < cand:
            best_def = cand
    
    #Couldn't find someone better that the best sampled. Return the best of the rest
    return best_def


if __name__ == '__main__':

    qt_cand = input('How many candidates do you want to generate? ')
    if int(qt_cand) > 1:
        #Generating a list of random elements. Let's pick qt_cand numbers in an universe of 1 to (qtc_cand*2).
        candidates = sample(list(range(1,int(qt_cand)*2)),int(qt_cand))

        #Select the best one using the 37% rule
        best_37 = opt_stop_37(candidates)
        print(f'The best one selected by the 37% rule is {best_37}')

How many candidates do you want to generate?  15000


The best one selected by the 37% rule is 29997


<br>Now let's run a batch of comparisions between the selections made by the 37% rule and a selection of the best candidate that runs through the whole list of candidates. This can be used to validate the table presented in the book.

In [5]:
def full_scan(candidates):
    return max(candidates)


qt_cand = input('How many candidates do you want to generate? ')
qt_comp = input('How many times do you want to compare the 37% selection with a full scan selection? ')

qt_right = int()
qt_wrong = int()
for comp in range(0,int(qt_comp)):
    #Generating a list of random elements. Let's pick qt_cand numbers in an universe of 1 to (qtc_cand*2).
    candidates = sample(list(range(1,int(qt_cand)*2)),int(qt_cand))
    
    #Select the best one using the 37% rule and then the really best using the full scan
    best_37 = opt_stop_37(candidates)  
    best_full = full_scan(candidates)
    
    if best_37 == best_full:
        qt_right += 1
    else:
        qt_wrong += 1
    
success_rate = qt_right / int(qt_comp) * 100
print(f'Out of {qt_comp} selections the 37% rule got {qt_right} right, missing {qt_wrong}. Success rate: {success_rate}')

How many candidates do you want to generate?  100000
How many times do you want to compare the 37% selection with a full scan selection?  10000


Out of 10000 selections the 37% rule got 3669 right, missing 6331. Success rate: 36.69
